# What is this project about?

Our data was provided by [Barun Kumar](https://www.kaggle.com/barun2104). It contains information on customers of a telecommunication company and whether they left the company or not. In this project we will see which types of customers are most likely to leave the company using classification.

## Import packages and data

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.graph_objects as pgo
import plotly.express as px

# Machine Learning
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Import data
df = pd.read_csv('../input/telecom-churn/telecom_churn.csv')

# How does our data look like? 

Let's have a look at the first five rows.

In [ ]:
df.head()

How many rows and columns do we have?

In [ ]:
df.shape

**Let's have a closer look at the variables**

The following table contains all variables with their description, data-type and the number of missing values. 

In [ ]:
# Create Table
var = {'Description': ['1 if customer cancelled service, 0 if not', 'number of weeks customer has had active account', '1 if customer recently renewed contract, 0 if not', '1 if customer has data plan, 0 if not', 'gigabytes of monthly data usage', 'number of calls into customer service', 'average daytime minutes per month', 'average number of daytime calls', 'average monthly bill', 'largest overage fee in last 12 months', 'average number of roaming minutes'],
       'Datatype': list(df.dtypes),
       '# of NaNs': list(df.isna().sum())}

df_var = pd.DataFrame(var, columns = ['Description', 'Datatype', '# of NaNs'], index = list(df.columns))
df_var

**This looks great!**

We don't have any categorical variables and no missing values. So our dataset is ready to be analysed. Of course the variable 'Churn' will be our target feature.

# Exploratory Data Analysis

Now we will see what insights and relationships can be discovered through analysing the variables. We will start with our target feature 'Churn'. 

## Target variable: 'Churn'

**How high is the proprotion of customers which cancelled the service?**

As described in the table above, this variable is '1' if the customer cancelled the service and '0' if not. 

In [ ]:
# Create new dataframe for better labeling of plots
df['churn'] = np.where(df['Churn'] == 1, 'cancelled', 'not cancelled')
df['contract renewal'] = np.where(df['ContractRenewal'] == 1, 'renewed', 'not renewed')
df['data plan'] = np.where(df['DataPlan'] == 1, 'data plan', 'no data plan')

# Create pie chart
labels = ['cancelled', 'not cancelled']
values = [(df.Churn == 1).sum(), (df.Churn == 0).sum()]
fig = pgo.Figure(data = [pgo.Pie(labels = labels, values = values, hole = .6, marker_colors = ['coral',                                                                                                                       
                                                                                               'cornflowerblue'])])
fig.show()

## Binary Features: 'ContractRenewal' and 'DataPlan'

The feature 'ContractRenewal' tells us wether the customer recently renewed his contract (1) or not (0). The feature 'DataPlan' is 1 if the customer had a data plan and 0 else.

We will see how many customers renewed their contract and how many had a data plan in this dataset. Additionaly we will use 'Churn' as a third variable in our plots to hopefully gain first insights of who is more likely to leave the company.

In [ ]:
# Create barchart for 'ContractRenewal'
sns.set_style("darkgrid")
df_plot = df.groupby(['churn', 'contract renewal']).size().reset_index().pivot(columns = 'churn', index = 'contract renewal', values = 0)
df_plot.plot(kind = 'bar', stacked = True, figsize = (10, 7))
plt.xticks(rotation = 0)

* In this dataset around 90% renewed their contract recently. 
* We can see that the proportion of customers who cancelled their contract is much higher in the group of customers who haven't renewed their contract recently. 
* In the group with a contract renewal the proportion is only around 10%, whereas in the other group it is more than one third.

In [ ]:
# Create barchart for 'DataPlan'
df_plot = df.groupby(['churn', 'data plan']).size().reset_index().pivot(columns = 'churn', index = 'data plan', values = 0)
df_plot.plot(kind = 'bar', stacked = True, figsize = (10, 7))
plt.xticks(rotation = 0)

* Only around 27% of customers have a data plan in this dataset.
* There is no significant difference in the proportion of cancelled contracts in these two groups.
* In the group with no data plan around 16% and in the other group 11% left the company.

## All non-binary features

Now we will look at the non-binary features and if they can help us identify the reason for leaving the company.

In [ ]:
# Set up figure
fig = plt.figure(figsize = (20,16))

# Violinplots
nbin_var = df[['AccountWeeks', 'DataUsage', 'CustServCalls', 'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee',
               'RoamMins']]
for i, v in enumerate(nbin_var):
    axes = fig.add_subplot(4, 2, i+1)
    sns.violinplot(x = 'churn', y = v, data = df, ax = axes)

* Half of the non-binary variables show no significant difference regarding the behavior of the two groups.
* The other half give us some interesting insights.

**DataUsage:**
* The median in both groups is around 0. This is most probably the case because as we already found out, only 27% have a data plan in this dataset.
* In the group of cancelled contracts the interquartile range is not nearly as large as in the other group. Also the above whisker is very small in comparison.
* This means that the data usage is more spread out for customers who then stayed in the company.

**CustServCalls:**
* Those who cancelled their service have one customer-service call more on average (median) than those who stayed in the company. 

**DayMins:**
* They also had higher average daytime minutes per month.

**MonthlyCharge:**
* The monthly charge was also higher on average in the group of customers who cancelled.

## Relationship: 'DataPlan' & 'DataUsage' & 'Churn'

We will now focus on other relationships and thereby try to better understand our data. We will start with plotting the data usage of customers with data plan vs customers without a data plan. We will keep 'Churn' as a third variable in the plot.

In [ ]:
# Create boxplots
sns.catplot(x = 'DataUsage', y = 'data plan', row = 'churn',
            kind = 'box', orient = 'h', height = 2.5, aspect = 4,
            data = df)

Of course, customers with a data plan had higher data usage on average than those without. The more valuable insights here are:
* Customers with a data plan who cancelled their service used more data on average than those who stayed in the company.
* 50% of those with no data plan who stayed in the company made use of data whereas only outliers of the group which left the company did so.

## Relationship: 'DataPlan' & 'MonthlyCharge' & 'Churn'

Now we will look at the differences in the monthly charge between costumers with and without a data plan.

In [ ]:
# Create boxplots
sns.catplot(x = 'MonthlyCharge', y = 'data plan', row = 'churn',
            kind = 'box', orient = 'h', height = 2.5, aspect = 4,
            data = df)

The monthly charge is also higher on average for those with a data plan. The more interesting fact about these boxplots is that the average monthly charge of the customers who had no data plan and left the company and was higher (around 62) than those who stayed in the company (around 47).

## Relationship: 'DataPlan' & 'DayMins' & 'Churn' 

Lastly let's have a look at the differences in their average daytime minutes per month.

In [ ]:
# Create boxplots
sns.catplot(x = 'DayMins', y = 'data plan', row = 'churn',
            kind = 'box', orient = 'h', height = 2.5, aspect = 4,
            data = df)

For the customers who stayed in the company the distribution of daytime minutes per month looks almost the same. For those who cancelled their service the difference between the group with and without a data plan is large. The average of those with a data plan is almost as high as the one of the customers who stayed in the company with 170 minutes. But for those who had no data plan is around 225 minutes. 

## Relationship: 'DataUsage' & 'MonthlyCharge' & 'Churn'

Of course the monthly charge of customers with high data usage will be higher on average. But by adding the variable 'Churn' as a third variable we could observe additional important insights.

In [ ]:
# Create scatterplot
plt.figure(figsize = (12, 8))
sns.scatterplot(data = df, x = 'DataUsage', y = 'MonthlyCharge', hue = 'churn')

As predicted, there is a positive correlation between data usage and monthly charge. What we can also observe is that most customers with low to no data usage but a high monthly charge left the company.

## Relationship: 'Daymins' & 'MonthlyCharge' & 'Churn' 

We will do the same for 'DayMins' and 'MonthlyCharge'.

In [ ]:
# Create scatterplot
plt.figure(figsize = (12, 8))
sns.scatterplot(data = df, x = 'DayMins', y = 'MonthlyCharge', hue = 'churn')

There is also a positive relationship between the average daytime minutes per month and the monthly charge. Here we can additionally observe one surprising fact. Almost all customers with a monthly charge between 60 and 80 and average daytime minutes per month above 200 left the company. The other leavers are well distributed in this scatterplot.

## Conclusion

After this analysis we now more about relationships of the variables and what kind of customers the company. We will now use Machine-Learning-Models to have a precise classification of the costumers in there leaving-behavior.

# Data Preparation

**Define target variable, features and split the data into train and test**

In [ ]:
# Define target and features
y = df['Churn']
X = df.iloc[:, 1:11]

# Stratified train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

**Resampling**

As we saw in our first pie chart the target feature 'churn' has a problem of imbalanced classes. It had 2850 customers who stayed in the company and 483 who left. The problem with imbalanced classes is that the data isn't easily seperable. We must make sacrifices to one class or the other.
That's why we will try out Upsampling and Downsampling our train-data prior to fitting our machine-learning-models. Then we will see which model and which resampling method worked best on our test-data.

In [ ]:
# Merge X_train and y_train
df_train = X_train.merge(y_train.to_frame(), left_index = True, right_index = True)

# Indicies of each class' observations
df_train0 = df_train[df_train['Churn'] == 0]
df_train1 = df_train[df_train['Churn'] == 1]

# Number of observations in each class
n_0 = len(df_train0)
n_1 = len(df_train1)

# Downsampling
df_train0_ds = resample(df_train0, replace = False, n_samples = n_1, random_state = 123)
df_train_ds = pd.concat([df_train0_ds, df_train1])
print("Downsampled Dataset Class Counts:", df_train_ds.Churn.value_counts(), sep = "\n")

# Upsampling
df_train1_us = resample(df_train1, replace = True, n_samples = n_0, random_state = 123)
df_train_us = pd.concat([df_train1_us, df_train0])
print("\nUpsampled Dataset Class Counts:", df_train_us.Churn.value_counts(), sep = "\n")

# Seperate y and X
y_train_ds = df_train_ds['Churn']
X_train_ds = df_train_ds.iloc[:, 0:10]

y_train_us = df_train_us['Churn']
X_train_us = df_train_us.iloc[:, 0:10]

# ML-Models

We will define a function which lets us use and compare multiple models without having to write the code multiple times. As performence metric we will use the normal accuracy score and "Area under the ROC-Curve" as it better represents the performance in a classification problem.

In [ ]:
# Set up function which uses an algo as input and returns the accuracy score and AUROC for the input data
def alg_fit(alg, X_train, y_train):
    
    # Model selection
    mod = alg.fit(X_train, y_train)
    
    # Prediction
    y_pred = mod.predict(X_test)
    
    # Accuracy Score
    acc_score = accuracy_score(y_test, y_pred)
    print("Accuracy Score: ", acc_score)
    
    # AUROC
    prob_y = mod.predict_proba(X_test)
    prob_y = [p[1] for p in prob_y]
    auroc_score = roc_auc_score(y_test, prob_y)
    print("AUROC Score:", auroc_score)

In [ ]:
# Set up function which uses function alg_fit and returns a comparison of the accuracy score and AUROC for imbalanced, downsampled and upsampled data
def compare_results(alg):
    # Unsampled Dataset
    print("IMBALANCED CLASSES")
    alg_fit(alg, X_train, y_train)

    # Downsampled majority class
    print("\nDOWNSAMPLED MAJORITY CLASS")
    alg_fit(alg, X_train_ds, y_train_ds)

    # Upsampled minority class
    print("\nUPSAMPLED MINORITY CLASS")
    alg_fit(alg, X_train_us, y_train_us)

## Logistic Regression 

Let's have a look at the first model and how it performs on the data with imbalanced classes, downsampled majority class and upsampled minority class.

In [ ]:
# Logistic Regression
compare_results(LogisticRegression(max_iter = 500))

It is ok that the accuracy metric is lower for the resampled data as it is expected. We will focus more on the AUROC-score as it is a better precision metric for our classification models. Here, upsampling the minority class yields the best result.

## Gaussian Naive Bayes

In [ ]:
# Gaussian Naive Bayes
compare_results(GaussianNB())

Here, the AUROC-score for the resampled data is even lower than with imbalanced classes. But still the precision is higher than for Logistic Regression.

## K-nearest Neighbors 

In [ ]:
# K-nearest Neighbors
compare_results(KNeighborsClassifier(n_neighbors = 3))

So the k-nearest neighbor didn't perform that good.

## Decision Tree 

In [ ]:
# Decision Tree Classifier
compare_results(DecisionTreeClassifier(random_state = 0, max_depth = 4))

We have a new accuracy high with the decision tree. 

## Random Forest 

In [ ]:
# Random Forest Classifier
rfc = RandomForestClassifier(random_state = 0, n_estimators = 50, max_depth = 5)
compare_results(rfc)

As forseeable Random Forest performed even better than a single decision tree.

## Gradient Boosting 

In [ ]:
# Gradient Boosting Classifier
gbc = GradientBoostingClassifier(random_state = 0, n_estimators = 50, max_depth = 3, 
                                 min_samples_leaf = 5)
compare_results(gbc)

Besides the increase in the AUROC-Score with downsampled majority class nothing has increased with Gradient-Boosting.

## XG-Boost 

In [ ]:
# XG-Boost Classifier
xgb = XGBClassifier(learning_rate = 0.15,
                    n_estimators = 1000,
                    max_depth = 20,
                    min_child_weight = 1,
                    gamma = 0.5,
                    subsample = 0.8,
                    colsample_bytree = 0.8,
                    reg_alpha = 0.03,
                    objective = 'binary:logistic',
                    nthread = 4,
                    scale_pos_weight = 1,
                    seed = 27)
compare_results(xgb)

The accuracy is higher but the AUROC-score of the Random-Forest Classifier is still the highest.

## Confusion Matrix, Recall and Precision

We will have a look at the confusion matrices of the last three models (Random Forest, Gradient Boosting & XGBoost) as they performed the best. Additionally, we are going to focus on the models with upsampled minority class as for all it yielded the highest AUROC Score.

In [ ]:
# List of best algos with parameter tuned
best_algs = [rfc, gbc, xgb]
names = ["RandomForestClassifier", "GradientBoostingClassifier", "XGBClassifier"]

# Return confusion matrices (absolute), precision and recall
j = 0
for i in best_algs:
    mod = i.fit(X_train_us, y_train_us)
    y_pred = mod.predict(X_test)
    con_mat = confusion_matrix(y_test, y_pred)
    prec = (con_mat[0][0])/(con_mat[0][0] + con_mat[0][1])
    rec = (con_mat[0][0])/(con_mat[0][0] + con_mat[1][0])
    print("\n", names[j], "Confusion Matrix:\n", con_mat, "\nPrecision: ", prec, "\nRecall: ", rec)
    j = j + 1

## Final Result

Our Random-Forest Model actually had the highest AUROC-Score and Recall. But the Accuracy and Precision of our XG-Boost Classifier outperformed the Random-Forest Model.

## Feature Importance

We will use the XG-Boost Classifier to detect the most important features.

In [ ]:
# Feature importance
df_feature_importance = pd.DataFrame(xgb.feature_importances_, index = X_train_us.columns, 
                                     columns = ['feature importance']).sort_values('feature importance', 
                                                                                   ascending = False)
df_feature_importance